In [62]:
## packages to handle array and datafram
import numpy as np
import pandas as pd

## Sklearn packages
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [63]:
## Reading the dat files
year_train, indicator_train = np.loadtxt("train.dat", usecols=(0,1), unpack=True)
year_test, indicator_test = np.loadtxt("test.dat", usecols=(0,1), unpack=True)

In [64]:
year_train.reshape(-1,1)

array([[1998.],
       [2008.],
       [1975.],
       [2004.],
       [1980.],
       [2002.],
       [2007.],
       [1979.],
       [1992.],
       [1982.],
       [2017.],
       [1991.],
       [1981.],
       [1977.],
       [1976.],
       [1973.],
       [2001.],
       [1999.],
       [1974.],
       [2003.],
       [2005.],
       [1993.],
       [1988.],
       [1994.],
       [1997.],
       [2018.],
       [1989.],
       [1987.],
       [2012.],
       [2019.],
       [2000.],
       [1971.],
       [2006.],
       [1970.],
       [2021.],
       [2014.],
       [1986.],
       [1984.],
       [1978.],
       [1996.],
       [2015.],
       [2020.]])

In [65]:
train = pd.DataFrame([year_train,indicator_train])
train = train.T
train.columns = ["x","y"]
test = pd.DataFrame([year_test,indicator_test])
test = test.T
test.columns = ["x","y"]

In [66]:
train["scaled_x"] = (train.x - train.x.mean())/train.x.std()

In [67]:
test["scaled_x"] = (test.x - test.x.mean())/test.x.std()

In [68]:
def crossvals_poly(alpha,d):
    ## crossvalidation
    k = 6
    cv_method = KFold(n_splits=k, shuffle=False, random_state=None)
    scores = []
    x_scaled = np.array(train["scaled_x"]).reshape(-1,1)
    y = np.array(train.y).reshape(-1,1)
    for i, (train_idx, test_idx) in enumerate(cv_method.split(x_scaled, y)):
        #splitting train and test
        
        X_train, y_train = x_scaled[train_idx], y[train_idx]
        X_test, y_test = x_scaled[test_idx], y[test_idx]
        
        
        poly = PolynomialFeatures(degree=d)
        X_train_scaling_trans = poly.fit_transform(X_train)
        X_test_scaling_trans = poly.fit_transform(X_test)

        # Ridge Regression
        identity_matrix = np.identity(d+1)
        w = (np.linalg.inv(X_train_scaling_trans.T @ X_train_scaling_trans + alpha * identity_matrix) @ X_train_scaling_trans.T @ y_train)
        y_hat = X_test_scaling_trans @ w

        #y_denormalize = (y_hat * Std_train) + Mean_train

        score1 = (y_hat - y_test)**2
        score2 = score1.mean()
        score = math.sqrt(score2)

        scores.append(score)
    
    return np.array(scores).mean()

In [69]:
degree_array = []
for d in range(13):
    degree_array.append([d,crossvals_poly(0,d)])

In [70]:
degree_array

[[0, 1.0155605589711298],
 [1, 1.0835561617376985],
 [2, 0.775429312351414],
 [3, 0.7830016853814872],
 [4, 0.4934818442563906],
 [5, 0.5701350133901241],
 [6, 0.14235262539255328],
 [7, 0.18750847543324478],
 [8, 0.14631007432971915],
 [9, 0.23647291085945565],
 [10, 0.1648808008707255],
 [11, 0.6257835765341546],
 [12, 0.6869783465274861]]

In [71]:
lambda_val_ = [0,math.exp(-25),math.exp(-20),
             math.exp(-14),math.exp(-7),
             math.exp(-3),1,math.exp(7),
             math.exp(3)]
lambda_array = []
for alpha in lambda_val_:
    lambda_array.append([alpha,crossvals_poly(alpha,12)])

In [72]:
lambda_array

[[0, 0.6869783465274861],
 [1.3887943864964021e-11, 0.6869783216219808],
 [2.061153622438558e-09, 0.6869791335472825],
 [8.315287191035679e-07, 0.6873008674599229],
 [0.0009118819655545162, 0.40644187027699763],
 [0.049787068367863944, 0.4471778810356248],
 [1, 3.84220657023011],
 [1096.6331584284585, 63.098296953361285],
 [20.085536923187668, 30.16193152673507]]

In [93]:
x_scaled = np.array(train["scaled_x"]).reshape(-1,1)

X_scaled= np.array(test["scaled_x"]).reshape(-1,1)
y = np.array(train.y).reshape(-1,1)
d=6
poly = PolynomialFeatures(degree=d)
X__trans = poly.fit_transform(x_scaled)
X_test_trans = poly.fit_transform(X_scaled)
identity_matrix = np.identity(d+1)
w = (np.linalg.inv((X__trans.T @ X__trans) + (math.exp(-3) * identity_matrix)) @ X__trans.T @ y)
y_hat1 = X__trans @ w

In [94]:
math.sqrt((np.square(y_hat1.flatten()-np.array(train.y)).mean()))

0.16281526798810722

In [75]:
x_scaled

array([[ 0.20976874],
       [ 0.86725283],
       [-1.30244469],
       [ 0.60425919],
       [-0.97370264],
       [ 0.47276237],
       [ 0.80150442],
       [-1.03945105],
       [-0.18472172],
       [-0.84220582],
       [ 1.45898852],
       [-0.25047013],
       [-0.90795423],
       [-1.17094787],
       [-1.23669628],
       [-1.43394151],
       [ 0.40701397],
       [ 0.27551715],
       [-1.3681931 ],
       [ 0.53851078],
       [ 0.6700076 ],
       [-0.11897331],
       [-0.44771536],
       [-0.0532249 ],
       [ 0.14402033],
       [ 1.52473693],
       [-0.38196695],
       [-0.51346377],
       [ 1.13024647],
       [ 1.59048534],
       [ 0.34126556],
       [-1.56543833],
       [ 0.73575601],
       [-1.63118674],
       [ 1.72198216],
       [ 1.26174329],
       [-0.57921218],
       [-0.710709  ],
       [-1.10519946],
       [ 0.07827192],
       [ 1.3274917 ],
       [ 1.65623375]])

In [76]:
np.array(test.y)

array([65.88071, 66.39677, 67.12844, 65.43429, 65.79104, 66.28228,
       66.55062, 62.80697, 67.09466, 67.17076])